# DEAP

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, AvgPool1D, Dense, Conv1D, Flatten, Dropout, Input, BatchNormalization, GlobalMaxPool1D, MaxPool1D, SpatialDropout1D, GlobalAvgPool1D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler                                                      
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix 
from scipy import signal
import pickle as pkl
import gc

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint


In [7]:
sampling_rate = 128
window_size = 1280
overlap = 256
channel_len = 32
classes=2

bands = {'delta': [0.5/(sampling_rate/2), 4/(sampling_rate/2)], 'theta': [4/(sampling_rate/2), 8/(sampling_rate/2)], \
         'alpha': [8/(sampling_rate/2), 14/(sampling_rate/2)], 'beta': [14/(sampling_rate/2), 30/(sampling_rate/2)], \
         'gamma': [30/(sampling_rate/2), 50/(sampling_rate/2)]} # FIXED: 75 -> 50

## Loading Data

In [8]:
def load_data(eeg_band = None):

  eeg_signal = []
  valence = []
  arousal = []
  dominance = []
  subject = {} # FIXED: Add subject dict

  for i in range(32):
    subject[i] = "{0:02d}".format(i+1)

  for person in range(32):
    print('Person No.' + str(person))
    
    # EEG files address
    address = './eeg_data/DEAP/s'+subject[person]+'.dat'

    with open(address, 'rb') as file:
      data = pkl.load(file, encoding = 'latin1')

    eeg = data['data']
    label = data['labels']
    
    # Assigning classes
    label[label<5.5] = 0
    label[label>=5.5] = 1     

    val = label.T[0] # Valence label
    aro = label.T[1] # Arousal label
    dom = label.T[2] # Dominance label

    del data, label
    

    for i in range(40): # Iterating through 40 vidoes/trials

      sig = eeg[i].T
      v = val[i]
      a = aro[i]
      d = dom[i]

      sig = sig[:, :32] # Considering all 32 EEG channels
      
      num, den = signal.butter(4, bands[eeg_band], 'band') # Butterworth filter of order N = 4
      band_signal = signal.filtfilt(num, den, sig, axis=0)
      sig = band_signal
      del band_signal, num, den
      
      scaler = StandardScaler().fit(sig)
      scaled_sig = scaler.transform(sig)

      del sig
        
      # Segmenting into 10 seconds (1280 timesteps) windows with 2 seconds (256 timesteps) overlap
      start = 0
      while start + window_size < scaled_sig.shape[0]:
        eeg_signal.append(scaled_sig[start:start+window_size, :])

        valence.append(v)
        arousal.append(a)
        dominance.append(d)
        start += overlap # NOTE: Isn't 'window_size - overlap' right?

      del scaled_sig


  eeg_signal = np.asarray(eeg_signal, dtype = np.float32) # Using 32 bit floating point value to save memory
  valence = np.asarray(valence, dtype = np.int8)
  arousal = np.asarray(arousal, dtype = np.int8)
  dominance = np.asarray(dominance, dtype = np.int8)

  print(valence.shape, valence[valence == 0].shape, valence[valence == 1].shape)
  print(arousal.shape, arousal[arousal == 0].shape, arousal[arousal == 1].shape)
  print(dominance.shape, dominance[dominance == 0].shape, dominance[dominance == 1].shape)

  valence = np_utils.to_categorical(valence)
  arousal = np_utils.to_categorical(arousal)
  dominance = np_utils.to_categorical(dominance)

  print(valence)

    
  return (eeg_signal, valence, arousal, dominance)

In [10]:
FOLD = 10
eeg_band = 'gamma' # EEG band name

data, valence, arousal, dominance = load_data(eeg_band) # Loading processed data

nb_samples = data.shape[0] # Number of samples
print(nb_samples)
factor = nb_samples//FOLD # Kth fold by this factor
 
# shuffler = np.random.permutation(nb_samples) # Shuffling data

# data = data[shuffler]
# valence = valence[shuffler]
# arousal = arousal[shuffler]
# dominance = dominance[shuffler]

Person No.0
Person No.1
Person No.2
Person No.3
Person No.4
Person No.5
Person No.6
Person No.7
Person No.8
Person No.9
Person No.10
Person No.11
Person No.12
Person No.13
Person No.14
Person No.15
Person No.16
Person No.17
Person No.18
Person No.19
Person No.20
Person No.21
Person No.22
Person No.23
Person No.24
Person No.25
Person No.26
Person No.27
Person No.28
Person No.29
Person No.30
Person No.31
(34560,) (18711,) (15849,)
(34560,) (17766,) (16794,)
(34560,) (17739,) (16821,)
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
34560


## CNN Model

In [11]:
eeg_input = Input(shape = (window_size, channel_len), name='eeg_input') # Input layer

# CNN model
def get_CNN_design():
  x = Conv1D(filters = 32, kernel_size = 1, strides = 2, padding = 'valid', activation='relu', name='conv1')(eeg_input)
  x = Conv1D(filters = 32, kernel_size = 5, strides = 2, padding = 'valid', activation='relu', name='conv2')(x)
  x = AvgPool1D(pool_size=2, name='avg_pool1')(x)
  x = BatchNormalization(name='batch_norm1')(x)
  x = SpatialDropout1D(rate=0.0625, name = 'spatial_dropout1')(x)
  x = Conv1D(filters = 64, kernel_size = 5, strides = 2, padding = 'valid', activation='relu', name='conv3')(x)
  x = Conv1D(filters = 64, kernel_size = 5, strides = 2, padding = 'valid', activation='relu', name='conv4')(x)
  x = AvgPool1D(pool_size=2, name='avg_pool2')(x)
  x = BatchNormalization(name='batch_norm2')(x)
  x = SpatialDropout1D(rate=0.125, name = 'spatial_dropout2')(x)
  x = Conv1D(filters = 128, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv5')(x)
  x = Conv1D(filters = 128, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv6')(x)
  x = AvgPool1D(pool_size=2, name='avg_pool3')(x)
  x = BatchNormalization(name='batch_norm3')(x)
  x = SpatialDropout1D(rate=0.25, name = 'spatial_dropout3')(x)
  x = Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv7')(x)
  x = Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'valid', activation='relu', name='conv8')(x)
  x = GlobalAvgPool1D(name='global_pool1')(x)
  x = BatchNormalization(name='batch_norm4')(x)
  x = Dropout(0.2)(x)
  x = Dense(64)(x)
  x = Activation('tanh')(x)
  x = Dense(8)(x)
  x = Activation('tanh')(x)
  x = Dropout(0.1)(x)
  return x

def get_model():
  x = get_CNN_design()

  out = Dense(classes, activation='softmax', name = 'output')(x) # Output layer

  model = Model(inputs=eeg_input, outputs=out) # Creating a model instance
  
  adam = Adam(lr=1e-3,decay=1e-5) # Adam optimizer
  
  model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['categorical_accuracy']) # Compiling model
  model.summary()
  
  return model

In [12]:
wandb.init(project="emosync")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gywls0724 (emosync). Use `wandb login --relogin` to force relogin


## Training

### Valence

In [13]:
val_res = {'accuracy': [], 'confusion_matrix': []}

gc.collect() # Garbage collecter
for i in range(FOLD):
  print('Fold: '+str(i))

  X_train = np.concatenate((data[0 : i*factor], data[(i+1)*factor : nb_samples])) # Training data
  X_test = data[i*factor : (i+1)*factor] # Testing data
  val_train = np.concatenate((valence[0 : i*factor], valence[(i+1)*factor : nb_samples])) # Valence training labels
  val_test = valence[i*factor : (i+1)*factor] # Valence testing labels

  gc.collect() # Garbage collecter
  model = get_model()
  model.fit(X_train, val_train, epochs = 100, batch_size = 1024, shuffle = True, callbacks=[wandb.keras.WandbCallback(labels=["Low", "High"])])

  acc = model.evaluate(X_test, val_test)
  print(acc)

  val_res['accuracy'].append(acc)

  pred = model.predict(X_test)
  val_res['confusion_matrix'].append(confusion_matrix(val_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter


# Dumping valence results
file = './eeg_data/DEAP_valence_' + eeg_band + '.pkl'

with open(file, 'wb') as f:
  pkl.dump(val_res, f)


Fold: 0


/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            10304 

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            1030

KeyboardInterrupt: 

In [7]:
for i in val_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places

52.37
49.19
44.27
54.92
55.09
48.58
42.53
38.05
42.51
53.91


In [8]:
for i in val_res['confusion_matrix']:
  print(i)

[[1631  232]
 [1414  179]]
[[ 745  740]
 [1016  955]]
[[ 719 1279]
 [ 647  811]]
[[1729  350]
 [1208  169]]
[[1580  553]
 [ 999  324]]
[[ 623  997]
 [ 780 1056]]
[[ 738 1179]
 [ 807  732]]
[[ 662 1255]
 [ 886  653]]
[[1188  486]
 [1501  281]]
[[1298  727]
 [ 866  565]]


### Arousal

In [9]:
aro_res = {'accuracy': [], 'confusion_matrix': []}

gc.collect() # Garbage collecter
for i in range(FOLD):
  print('Fold: '+str(i))
  
  X_train = np.concatenate((data[0 : i*factor], data[(i+1)*factor : nb_samples])) # Training data
  X_test = data[i*factor : (i+1)*factor] # Testing data
  aro_train = np.concatenate((arousal[0 : i*factor], arousal[(i+1)*factor : nb_samples])) # Arousal training labels
  aro_test = arousal[i*factor : (i+1)*factor] # Arousal testing labels

  gc.collect() # Garbage collecter
  model = get_model()
  model.fit(X_train, aro_train, epochs = 100, batch_size = 1024, shuffle = True)

  acc = model.evaluate(X_test, aro_test)
  print(acc)

  aro_res['accuracy'].append(acc)

  pred = model.predict(X_test)
  aro_res['confusion_matrix'].append(confusion_matrix(aro_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter


# Dumping arousal results
file = './eeg_data/DEAP_arousal_' + eeg_band + '.pkl'

with open(file, 'wb') as f:
  pkl.dump(aro_res, f)


Fold: 0


/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

...

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

In [10]:
for i in aro_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places

46.99
49.71
54.75
50.72
50.75
52.0
60.42
62.91
50.64
51.94


In [11]:
for i in aro_res['confusion_matrix']:
  print(i)

[[ 794 1258]
 [ 574  830]]
[[ 921 1266]
 [ 472  797]]
[[1237  545]
 [1019  655]]
[[ 679  887]
 [ 816 1074]]
[[1018  764]
 [ 938  736]]
[[ 477  927]
 [ 732 1320]]
[[ 549  882]
 [ 486 1539]]
[[ 765  882]
 [ 400 1409]]
[[ 753 1353]
 [ 353  997]]
[[1565  244]
 [1417  230]]


### Dominance

In [12]:
dom_res = {'accuracy':[], 'confusion_matrix':[]}

gc.collect() # Garbage collecter
for i in range(FOLD):
  print('Fold: '+str(i))

  X_train = np.concatenate((data[0 : i*factor], data[(i+1)*factor : nb_samples])) # Training data
  X_test = data[i*factor : (i+1)*factor] # Testing data
  dom_train = np.concatenate((dominance[0 : i*factor], dominance[(i+1)*factor : nb_samples])) # Dominance training labels
  dom_test = dominance[i*factor : (i+1)*factor] # Dominance testing labels

  gc.collect() # Garbage collecter
  model = get_model()
  model.fit(X_train, dom_train, epochs = 100, batch_size = 1024, shuffle = True)

  acc = model.evaluate(X_test, dom_test)
  print(acc)
    
  dom_res['accuracy'].append(acc)

  pred = model.predict(X_test)
  dom_res['confusion_matrix'].append(confusion_matrix(dom_test.argmax(1), pred.argmax(1)))
  gc.collect() # Garbage collecter


# Dumping dominance results
file = './eeg_data/DEAP_dominance_' + eeg_band + '.pkl'

with open(file, 'wb') as f:
  pkl.dump(dom_res, f)

Fold: 0


/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

...

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

/home/hyojinju/.conda/envs/hyojin_py37_tf25/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
eeg_input (InputLayer)       [(None, 1280, 32)]        0         
_________________________________________________________________
conv1 (Conv1D)               (None, 640, 32)           1056      
_________________________________________________________________
conv2 (Conv1D)               (None, 318, 32)           5152      
_________________________________________________________________
avg_pool1 (AveragePooling1D) (None, 159, 32)           0         
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 159, 32)           128       
_________________________________________________________________
spatial_dropout1 (SpatialDro (None, 159, 32)           0         
_________________________________________________________________
conv3 (Conv1D)               (None, 78, 64)            103

KeyboardInterrupt: 

In [ ]:
for i in dom_res['accuracy']:
  print(round(i[1]*100, 2)) # Rounding off to two decimal places

In [ ]:
for i in dom_res['confusion_matrix']:
  print(i)